In [2]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np

preprocess_list = [
    lambda img: img.astype("float32")/255,
    mx.image.ForceResizeAug((224, 224)),
    mx.image.ColorNormalizeAug(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    lambda img: nd.transpose(img,(2,0,1))
]

def data_transform(img, label):
    for f in preprocess_list:
        img = f(img)
    return img,label

In [4]:
import os

data_root_dir = '/home/yansheng/kaggle_dog'
output_dir = 'train_valid_test'

input_str = os.path.join(data_root_dir, output_dir)
batch_size = 128

# 读取原始图像文件。flag=1说明输入图像有三个通道（彩色）。
train_ds = vision.ImageFolderDataset(input_str + '/train', flag=1,
                                     transform=data_transform)
valid_ds = vision.ImageFolderDataset(input_str + '/valid', flag=1,
                                     transform=data_transform)
train_valid_ds = vision.ImageFolderDataset(input_str + '/train_valid',
                                           flag=1, transform=data_transform)
test_ds = vision.ImageFolderDataset(input_str + '/test', flag=1,
                                     transform=data_transform)

loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_ds, batch_size, shuffle=True, last_batch='keep')
test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep')

In [5]:
ctx = mx.gpu()
resnet50_v2 = mx.gluon.model_zoo.vision.resnet50_v2(pretrained=True, ctx=ctx)

In [8]:
def extract_features(net, data, ctx):
    rst_X, rst_y = [], []
    for X, y in data:
        Xi = net.features(X.as_in_context(ctx))
        rst_X.append(Xi.asnumpy())
        rst_y.append(y.asnumpy())
    return np.concatenate(rst_X, axis=0), np.concatenate(rst_y, axis=0)

In [9]:
X_train_resnet50_v2, y_train = extract_features(resnet50_v2, train_valid_data, ctx)
X_test_resnet50_v2, _ = extract_features(resnet50_v2, test_data, ctx)

In [11]:
import h5py

with h5py.File(os.path.join(data_root_dir, 'resnet50_v2_pretrained_Xy.h5'), 'w') as f:
    f['X_train_resnet50_v2'] = X_train_resnet50_v2
    f['X_test_resnet50_v2'] = X_test_resnet50_v2
    f['y_train'] = y_train